In [13]:
import pandas as pd
import networkx as nx
import matplotlib
import pickle
import numpy as np
import io

from IPython.display import display, HTML

from utils import *

In [14]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

dpi = 330
matplotlib.rcParams['figure.dpi']= dpi

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
TRAIN_SPLIT = 0.8

In [16]:
data = pd.read_csv("~/Downloads/beer_reviews.csv") 

In [17]:
data.head()

,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
0,10325,Vecchio Birraio,1234817823,1.5,2.0,2.5,stcules,Hefeweizen,1.5,1.5,Sausa Weizen,5.0,47986
1,10325,Vecchio Birraio,1235915097,3.0,2.5,3.0,stcules,English Strong Ale,3.0,3.0,Red Moon,6.2,48213
2,10325,Vecchio Birraio,1235916604,3.0,2.5,3.0,stcules,Foreign / Export Stout,3.0,3.0,Black Horse Black Beer,6.5,48215
3,10325,Vecchio Birraio,1234725145,3.0,3.0,3.5,stcules,German Pilsener,2.5,3.0,Sausa Pils,5.0,47969
4,1075,Caldera Brewing Company,1293735206,4.0,4.5,4.0,johnmichaelsen,American Double / Imperial IPA,4.0,4.5,Cauldron DIPA,7.7,64883


In [18]:
count_df = data.groupby('beer_name').size().reset_index(name='count')

In [19]:
size = len(data)

In [20]:
train_data = data[:int(size * TRAIN_SPLIT)]

In [21]:
graph = build_graph(train_data, 20)

In [22]:
print("Número de nodos: "+str(graph.number_of_nodes()))
print("Número de aristas: "+str(graph.number_of_edges()))

Número de nodos: 36856
Número de aristas: 2315895


In [23]:
index_data = data.set_index(['review_profilename', 'beer_beerid'])

In [10]:
len(index_data)

1586614

In [11]:
index_data = index_data[~index_data.index.duplicated(keep='last')]

In [12]:
index_data = index_data.drop(['brewery_name'], axis=1)
index_data = index_data.drop(['beer_name'], axis=1)
index_data = index_data.drop(['review_time'], axis=1)
# We already use this as weight
index_data = index_data.drop(['review_overall'], axis=1)

In [13]:
edge_attrs = index_data.to_dict(orient='index')

In [14]:
nx.set_edge_attributes(graph, edge_attrs)

In [15]:
graph.edges()['fodeeoz', 436]

{'weight': 3.0,
 'review_aroma': 2.0,
 'review_appearance': 3.0,
 'review_palate': 2.5,
 'review_taste': 2.5,
 'brewery_id': 163,
 'beer_style': 'Light Lager',
 'beer_abv': 3.5}

In [16]:
index_data.head()

brewery_id  review_aroma  review_appearance  \
review_profilename beer_beerid                                                
stcules            47986             10325           2.0                2.5   
                   48213             10325           2.5                3.0   
                   48215             10325           2.5                3.0   
                   47969             10325           3.0                3.5   
johnmichaelsen     64883              1075           4.5                4.0   

                                                    beer_style  review_palate  \
review_profilename beer_beerid                                                  
stcules            47986                            Hefeweizen            1.5   
                   48213                    English Strong Ale            3.0   
                   48215                Foreign / Export Stout            3.0   
                   47969                       German Pilsener            2.5   
johnmichaelsen     64883        American Double / Imperial IPA            4.0   

                                review_taste  beer_abv  
review_profilename beer_beerid                          
stcules            47986                 1.5       5.0  
                   48213                 3.0       6.2  
                   48215                 3.0       6.5  
                   47969                 3.0       5.0  
johnmichaelsen     64883                 4.5       7.7

In [17]:
beer_attrs = data[['beer_beerid', 'beer_style', 'brewery_id']]
beer_attrs = beer_attrs.set_index(['beer_beerid'])
beer_attrs = beer_attrs[~beer_attrs.index.duplicated(keep='last')]

In [18]:
beer_attrs.head()

,beer_style,brewery_id
beer_beerid,,
47986,Hefeweizen,10325
48213,English Strong Ale,10325
48215,Foreign / Export Stout,10325
47969,German Pilsener,10325
64883,American Double / Imperial IPA,1075


In [19]:
len(beer_attrs)

66055

In [20]:
beer_attrs = beer_attrs.to_dict(orient='index')

In [21]:
nx.set_node_attributes(graph, beer_attrs)

In [22]:
graph.nodes()[436]

{'beer_style': 'Light Lager', 'brewery_id': 163, 'beer_abv': 3.5}

In [23]:
index_data = index_data.drop(['brewery_id'], axis=1)
index_data = index_data.drop(['beer_style'], axis=1)

In [24]:
graph = build_graph(data, 20, debug=True)

Found 14806 duplicates.
Attrs for edge "fodeeoz", 436:
{'weight': 3.0, 'review_aroma': 2.0, 'review_appearance': 3.0, 'review_palate': 2.5, 'review_taste': 2.5}
Example attrs for node 436:
{'beer_style': 'Light Lager', 'brewery_id': 163, 'beer_abv': 3.5}
Adding 1682942 edges from beer similarity based on styles.
Adding 175666 edges from beer similarity based on breweries.


history_u_lists: diccionario item-> usuarios que lo compraron
history_ur_lists: diccionario item-> ratings (del usuario que lo compro) (igual largo que el anterior)

history_v_lists: diccionario usuario-> items que compro
history_vr_lists: diccionario usuario-> ratin para el item que compro (igual largo que el anterior)

train_u: lista de ids de items para train
train_v: lista de ids de usuarios para train
train_r: lista de ratings para train
Todo esto tiene que coincidir por indice (HORRIBLE). Los largos de los 3 coinciden. Estan codificando (de la peor forma) tripletas usuario-item-rating. 

social_adj_lists: diccionario usuario: set de usuarios conectados

ratings_list: diccionario con rating posible-> indice - {2.0: 0, 1.0: 1, 3.0: 2, 4.0: 3, 2.5: 4, 3.5: 5, 1.5: 6, 0.5: 7}



In [257]:
dir_data = './GraphRec-WWW19/data/toy_dataset'

path_data = dir_data + ".pickle"
data_file = open(path_data, 'rb')
history_u_lists, history_ur_lists, history_v_lists, history_vr_lists, train_u, train_v, train_r, test_u, test_v, test_r, social_adj_lists, ratings_list = pickle.load(
    data_file)

In [164]:
split_data = np.array([[user, item, list(graph.get_edge_data(user, item).values())] for user, item in graph.edges() if isinstance(user, str) and isinstance(item, int)])

/Users/fabrizio.albertoni/Envs/beer-rec/lib/python3.7/site-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  """Entry point for launching an IPython kernel.


In [167]:
np.random.shuffle(split_data)

In [172]:
train_u, train_v, train_r = np.split(split_data, 3, axis=1)

In [187]:
len(train_u)

14091

In [ ]:
history_u_lists, history_ur_lists, history_v_lists, history_vr_lists, train_u, train_v, train_r, test_u, test_v, test_r, item_adj_lists = decompose_graph(graph)

In [25]:
data = decompose_graph(graph)

1
2
2.1
3
4
5


/Users/fabrizio.albertoni/Projects/beer-recommender/utils.py:166: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  for user, item in graph.edges()


6


/Users/fabrizio.albertoni/Envs/beer-rec/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


Done transforming. 
7


In [27]:
with open('data.pkl', 'wb') as handle:
    pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [264]:
len(history_v_lists)

1941

In [268]:
max(train_v)

1940

In [ ]:
# TODO El codigo asume que los ids son consecutivos
# Use this? https://networkx.org/documentation/networkx-1.10/reference/generated/networkx.relabel.convert_node_labels_to_integers.html#networkx.relabel.convert_node_labels_to_integers

In [43]:
data = history_v_lists, history_vr_lists, history_u_lists, history_ur_lists, train_v, train_u, train_r, test_v, test_u, test_r, item_adj_lists

In [31]:
from run_GraphRec_example import run

In [32]:
with open('data.pkl', 'rb') as handle:
    data = pickle.load(handle)

In [ ]:
run(data)

31883
9186
[1,     0] loss: 0.147, The best rmse/mae: 9999.000000 / 9999.000000
[1,   100] loss: 11.790, The best rmse/mae: 9999.000000 / 9999.000000
[1,   200] loss: 6.588, The best rmse/mae: 9999.000000 / 9999.000000
[1,   300] loss: 3.213, The best rmse/mae: 9999.000000 / 9999.000000
[1,   400] loss: 2.077, The best rmse/mae: 9999.000000 / 9999.000000
[1,   500] loss: 1.624, The best rmse/mae: 9999.000000 / 9999.000000
